In [1]:
from pyspark.sql import SparkSession
import glob
import numpy as np
import pandas as pd
import os
import concurrent.futures
from tqdm import tqdm

In [2]:
#Set Environment accordingly
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home/"
os.environ['SPARK_HOME'] = "/Users/simran/Downloads/spark-3.5.5-bin-hadoop3/"

In [3]:
!wget "https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_price/full_history.zip"
!unzip "./full_history.zip" -d "./data/"
!rm -rf "./full_history.zip"
!rm -rf "./data/__MACOSX"

--2025-05-03 14:14:19--  https://huggingface.co/datasets/Zihan1004/FNSPID/resolve/main/Stock_price/full_history.zip
Resolving huggingface.co (huggingface.co)... 2600:9000:2801:3c00:17:b174:6d00:93a1, 2600:9000:2801:6a00:17:b174:6d00:93a1, 2600:9000:2801:d400:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:2801:3c00:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a0/19/a01967ba742831f4b6903f146b4f5d9a3d2eeeefa619387e8cc87df2d889332c/03da4fce7ebea90d5715ba3501773d410ae663b617027b338ef000a9955dab91?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27full_history.zip%3B+filename%3D%22full_history.zip%22%3B&response-content-type=application%2Fzip&Expires=1746299660&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NjI5OTY2MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2EwLzE5L2EwMTk2N2JhNzQyODMxZjRiNjkwM2YxNDZiN

In [4]:
directory_path = "./data/full_history"
file_pattern = "*.csv"

In [5]:
#Preprocess Data to ensure consistent schema with spark
desired_order = ['date','volume','open','high','low','close','adj close']
new_order = ['date','volume','open','high','low','close','adj close','StockName']
def amendCSV(filename):
  df = pd.read_csv(filename, usecols=desired_order)
  stockName = filename
  df['StockName'] = stockName.split("/")[-1].split(".csv")[0]  # Add filename column
  df.to_csv(filename,index=False,columns=new_order)
  return 1

all_files = glob.glob(os.path.join(directory_path, file_pattern), recursive=True)
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
  result = list(tqdm(executor.map(amendCSV, all_files), total = len(all_files)))
print('All done.')

100%|██████████| 7693/7693 [01:59<00:00, 64.63it/s]

All done.


In [6]:
!rm "./data/full_history/TRUE.csv"